In [1]:
import ee
import math
import geemap # Assuming geemap is installed

# --- Configuration ---
GCP_PROJECT = 'mapbiomas-solos-workspace'
AOI_ASSET_ID = 'projects/mapbiomas-workspace/AUXILIAR/brasil_2km'
SOIL_SAMPLES_ID = 'projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/matriz-v010_0_10cm'
MAPBIOMAS_COLLECTION_ID = 'projects/mapbiomas-workspace/COLECAO9/integracao'
OUTPUT_SCALE = 30  # meters
MAX_DISTANCE_METERS = 200000 # 200 km (Adjust if needed for visualization range)



In [2]:
# --- 1. Initialize Earth Engine ---
ee.Initialize(project=GCP_PROJECT)
print(f"EE initialized with project: {GCP_PROJECT}")

EE initialized with project: mapbiomas-solos-workspace


In [3]:
# --- 2. Define Area of Interest ---
print(f"Loading AOI: {AOI_ASSET_ID}")
aoi_asset = ee.FeatureCollection(AOI_ASSET_ID)
aoi_geometry = aoi_asset.union(1) # Merge features into a single geometry

Loading AOI: projects/mapbiomas-workspace/AUXILIAR/brasil_2km


In [4]:
# --- 3. Load and Filter Soil Samples ---
print(f"Loading Soil Samples: {SOIL_SAMPLES_ID}")
soil_points_all = ee.FeatureCollection(SOIL_SAMPLES_ID)
soil_points = soil_points_all.filterBounds(aoi_geometry)
point_count = soil_points.size().getInfo() # Get count after filtering
print(f"Found and filtered {point_count} points within AOI.")

# --- 3b. Visualize Filtered Points ---
if point_count > 0:
    print("\nPreparing visualization of filtered points on AOI...")
    Map_points = geemap.Map()
    Map_points.centerObject(aoi_geometry, 5) # Center on AOI
    Map_points.addLayer(ee.Image().paint(aoi_geometry, 0, 2), {'palette': '000000'}, 'AOI Boundary')
    Map_points.addLayer(soil_points, {'color': '0000FF', 'pointSize': 1}, 'Filtered Soil Samples')
    Map_points.addLayerControl()
    print("Displaying filtered points map...")
    display(Map_points)
    print("--- End of points visualization ---")
else:
    print("\nSkipping points visualization as no points were found within the AOI.")


# --- Add constant property (only if points exist) ---
if point_count > 0:
    def add_constant(feat):
      return feat.set({'constant': 1})
    soil_points_with_constant = soil_points.map(add_constant)
else:
    soil_points_with_constant = ee.FeatureCollection([])

Loading Soil Samples: projects/mapbiomas-workspace/SOLOS/AMOSTRAS/MATRIZES/granulometry/matriz-v010_0_10cm
Found and filtered 19864 points within AOI.

Preparing visualization of filtered points on AOI...
Displaying filtered points map...


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…

--- End of points visualization ---


In [5]:
# --- 4. Get Target Projection ---
print(f"\nGetting projection from MapBiomas Collection 9: {MAPBIOMAS_COLLECTION_ID}")
# Load the collection and get the first image to extract projection
# Note: Ensure this asset ID points to an Image or ImageCollection
mapbiomas_ref_image = ee.ImageCollection(MAPBIOMAS_COLLECTION_ID).first()
# Check if the reference image loaded successfully
if mapbiomas_ref_image is None:
     print(f"ERROR: Could not load reference image from {MAPBIOMAS_COLLECTION_ID}. Check asset ID and permissions.")
     # Handle error - maybe exit or try a fallback CRS
     target_crs = 'EPSG:4326' # Fallback CRS
     print(f"Warning: Using fallback CRS: {target_crs}")
else:
    target_crs = mapbiomas_ref_image.projection().crs()
    print(f"Target CRS from Collection 9: {target_crs.getInfo()}")



Getting projection from MapBiomas Collection 9: projects/mapbiomas-workspace/COLECAO9/integracao
Target CRS from Collection 9: EPSG:4326


In [7]:
# --- 5. Prepare Image for Distance Calculation ---
if point_count > 0:
    print("Rasterizing points...")
    point_presence_image = soil_points_with_constant.reduceToImage(
        properties=['constant'],
        reducer=ee.Reducer.sum().unweighted()
    ).reproject(
        crs=target_crs, # Use projection from Collection 9
        scale=OUTPUT_SCALE
    ).unmask(0)
    source_image = point_presence_image.gt(0).Not()
    print("Rasterization complete.")
# Print information about the rasterized image
print(f"Rasterized image info: {source_image.getInfo()}")
# Plot the rasterized image
Map_raster = geemap.Map()
Map_raster.centerObject(aoi_geometry, 5) # Center on AOI
Map_raster.addLayer(source_image.clip(aoi_geometry), {'palette': '0000FF'}, 'Rasterized Points')
Map_raster.addLayerControl()
print("Displaying rasterized points map...")
display(Map_raster)
print("--- End of rasterized points visualization ---")

Rasterizing points...
Rasterization complete.
Rasterized image info: {'type': 'Image', 'bands': [{'id': 'sum', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [0.00026949458523585647, 0, 0, 0, -0.00026949458523585647, 0]}]}
Displaying rasterized points map...


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…

--- End of rasterized points visualization ---


In [ ]:
# --- 6. Calculate Distance ---
print("Calculating distance to nearest neighbor...")
neighborhood_pixels = math.ceil(MAX_DISTANCE_METERS / OUTPUT_SCALE)
distance_image_multiband = source_image.fastDistanceTransform(
    neighborhood=neighborhood_pixels,
    units='meters'
)
distance_image = distance_image_multiband.select('distance')

# *** PRINT INFO ABOUT distance_image (NEW) ***
print("\n--- Info about 'distance_image' (before clipping) ---")
try:
    band_names = distance_image.bandNames().getInfo()
    print(f"Band Names: {band_names}")
    # The projection *should* match target_crs used in reproject before fastDistanceTransform
    print(f"Expected CRS: (Matches target_crs from Step 4)")
    print(f"Expected Scale (meters): {OUTPUT_SCALE}")
    # Note: Getting exact projection/scale info with .projection().getInfo() can be slow
    # and might trigger errors if the image is complex, so we state expectations.
except Exception as e:
    print(f"Could not retrieve full info: {e}")
print("----------------------------------------------------")
# *** END PRINT INFO ***

Calculating distance to nearest neighbor...

--- Info about 'distance_image' (before clipping) ---
Band Names: ['distance']
Expected CRS: (Matches target_crs from Step 4)
Expected Scale (meters): 30
----------------------------------------------------


In [49]:
# --- 7. Clip Result ---
print("Clipping result to AOI...")
distance_image_clipped = distance_image.clip(aoi_geometry)

Clipping result to AOI...


In [46]:
# --- 8. Visualize DNN Map ---

# Ensure geemap is imported if running this block standalone
import geemap

# Check if distance_image_clipped likely exists before trying to visualize
# (Assumes it was calculated in prior steps if point_count > 0)
if 'distance_image_clipped' in locals():
    print("\nPreparing final DNN map visualization...")
    # Create a new map object for the final result
    Map_dnn = geemap.Map()
    # Center the map on the Area of Interest
    Map_dnn.centerObject(aoi_geometry, 5)

    # Define visualization parameters for the distance map
    dnn_vis_params = {
      'min': 0,
      # Use the configured max distance for the color scale limit
      'max': MAX_DISTANCE_METERS,
      # Define the color palette (e.g., blue=near, red=far)
      'palette': ['0000FF', '00FFFF', 'FFFF00', 'FF0000']
    }
    print(f"Visualizing distances from 0 to {dnn_vis_params['max']} meters.")

    # Add the calculated distance layer to the map
    Map_dnn.addLayer(
        distance_image_clipped,
        dnn_vis_params,
        f'Distance to Nearest Soil Sample (m)' # Layer name
    )
    # Add the AOI boundary outline for context
    Map_dnn.addLayer(
        ee.Image().paint(aoi_geometry, 0, 2), # Paint outline
        {'palette': '000000'}, # Black color for outline
        'AOI Boundary' # Layer name
    )
    # Add map layer controls (optional)
    Map_dnn.addLayerControl()

    # Display the map in the Jupyter Notebook output
    print("Displaying final DNN map...")
    display(Map_dnn)

else:
    # This part would typically execute if point_count was 0 in the full script
    print("\nSkipping DNN map visualization because the distance image was not generated (likely no input points found).")


Preparing final DNN map visualization...
Visualizing distances from 0 to 200000 meters.
Displaying final DNN map...


Map(center=[-10.62347807211283, -53.19353084531423], controls=(WidgetControl(options=['position', 'transparent…